In [2]:
class Registry(object):
    def __init__(self):
        self._functions = []

    def regster(self, decorated):
        self._functions.append(decorated)
        return decorated
    
    def run_all(self, *args, **kwargs):
        return_values = []
        for func in self._functions:
            return_values.append(func(*args, **kwargs))
        return return_values


a = Registry()
b = Registry()


@a.regster
def foo(x=3):
    return x


@b.regster
def foo(x=4):
    return x


@a.regster
@b.regster
def foo(x=5):
    return x

a.run_all()

[3, 5]

In [3]:
b.run_all()

[4, 5]

In [3]:
def require_ints(decorated):
    def inner(*args, **kwargs):
        for arg in args:
            if not isinstance(arg,int):
                raise TypeError("%d only " %decorated.__name__)
        return decorated(*args,**kwargs)
    return inner


@require_ints
def foo(x, y):
    """return the sum of x and y"""
    return x+y

help(foo)

Help on function inner in module __main__:

inner(*args, **kwargs)



In [5]:
import functools


def require_ints(decorated):
    @functools.wraps(decorated)
    def inner(*args, **kwargs):
        for arg in args:
            if not isinstance(arg,int):
                raise TypeError("%d only " %decorated.__name__)
        return decorated(*args,**kwargs)
    return inner


@require_ints
def foo(x, y):
    """return the sum of x and y"""
    return x+y

help(foo)

Help on function foo in module __main__:

foo(x, y)
    return the sum of x and y



In [6]:
import functools
import logging
import time


def logged(method):
    @functools.wraps(method)
    def inner(*args, **kwargs):
        start = time.time()
        result = method(*args, **kwargs)
        end = time.time()
        delta = end - start
        logger = logging.getLogger('decorator.logged')
        logger.warn('called method %s at %.2f; execution time %.2f,seconds; result %r' %
                    (method.__name__, start, delta, result))

        return result
    return inner


@logged
def sleep_and_return(re_value):
    time.sleep(2)
    return re_value


sleep_and_return(42)

D:\Users\a\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
called method sleep_and_return at 1541407904.76; execution time 2.00,seconds; result 42


42

In [7]:
import json


class JSONoutputError(Exception):
    def __init__(self, mes):
        self._message = mes
        
    def __str__(self):
        return self._message
    

def json_output(indent=None, sort_keys=False):    
    def actual_decorator(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            try:
                result = func(*args, **kwargs)
                
            except JSONoutputError as ex:
                result = {
                    "status": 'error',
                    "message": str(ex)
                }
            return json.dumps(result, indent=indent, sort_keys=sort_keys)
        return inner
    return actual_decorator

In [8]:
@json_output(indent=3)
def do_nothing():
    return {'status': 'done'}

do_nothing()

'{\n   "status": "done"\n}'

In [9]:
def json_output(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        try:
            result = func(*args, **kwargs)

        except JSONoutputError as ex:
            result = {
                "status": 'error',
                "message": str(ex)
            }
        return json.dumps(result)
    return inner

In [10]:
@json_output
def do_nothing1():
    return {'status': 'done'}

do_nothing1()

'{"status": "done"}'

In [11]:
import json


class JSONoutputError(Exception):
    def __init__(self, mes):
        self._message = mes
        
    def __str__(self):
        return self._message
    

def json_output(decorated_=None, indent=None, sort_keys=False):
    if decorated_ and (indent or sort_keys):
        raise RuntimeError("inexpected arguments")
        
    def actual_decorator(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            try:
                result = func(*args, **kwargs)
                
            except JSONoutputError as ex:
                result = {
                    "status": 'error',
                    "message": str(ex)
                }
            return json.dumps(result, indent=indent, sort_keys=sort_keys)
        return inner
    if decorated_:
        return actual_decorator(decorated_)
    else:
        return actual_decorator

In [12]:
@json_output
def do_nothing1():
    return {'status': 'done'}

do_nothing1()

'{"status": "done"}'

In [13]:
@json_output(indent=3)
def do_nothing():
    return {'status': 'done'}

do_nothing()

'{\n   "status": "done"\n}'

In [14]:
def sort_by_creation_time(cls):
    original_init = cls.__init__

    @functools.wraps(original_init)
    def new_init(self, *args, **kwargs):
        original_init(self, *args, **kwargs)
        self._creation_time = time.time()
    cls.__init__ = new_init

    cls.__lt__ = lambda self, other: self._creation_time < other._creation_time
    cls.__gt__ = lambda self, other: self._creation_time > other._creation_time
    return cls


@sort_by_creation_time
class Sortable(object):
    def __init__(self, identifier):
        self._identifier = identifier
    
    def __repr__(self):
        return self._identifier

first = Sortable('first')
second = Sortable("2")
third = Sortable("3")
sortables = [first, second, third]

sorted(sortables)

[first, 2, 3]

In [20]:
class Task(object):
    def run(self, *args, **kwargs):
        raise NotImplementedError("subclasses must implement 'run'")
    
    def identify(self):
        return "i am a task"
    

def task(decorated):
    class Tasksubclass(Task):
        def run(self, *args, **kwargs):
            return decorated(*args, **kwargs)
    return Tasksubclass


@task
def foo():
    return 2+2

f = foo()
f.run()

4

In [21]:
f.identify()

'i am a task'

In [34]:
class Task(object):
    def run(self, *args, **kwargs):
        raise NotImplementedError("subclasses must implement 'run'")
    
    def identify(self):
        return "i am a task"

    def __call__(self, *args, **kwargs):
        return self.run(self, *args, **kwargs)


def task(decorated):
    class Tasksubclass(Task):
        def run(self, *args, **kwargs):
            return decorated()
    return Tasksubclass()


@task
def foo():
    return 2 + 2

foo.identify()

'i am a task'

In [35]:
foo()

4

__main__.task.<locals>.Tasksubclass